In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine

In [3]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [4]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [5]:
TABLE = 'flats_table'
SQL = f'select * from {TABLE}'
data = pd.read_sql(SQL, dst_conn)

In [6]:
data.head()

,id,building_id,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,price,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,14971,13921,9,8.5,21.000000,1,false,false,41.000000,7600000.0,1983,4,55.797527,37.459946,2.64,95,9,true
1,14972,10161,8,8.8,32.599998,2,false,false,49.000000,8990000.0,1972,6,55.676533,37.676754,2.64,84,12,true
2,14973,7378,2,9.5,21.100000,1,false,false,35.099998,6500000.0,1967,6,55.759617,37.801781,2.64,84,12,true
3,14974,10856,10,10.0,20.000000,1,false,false,38.000000,8900000.0,1974,4,55.737293,37.416267,2.48,111,14,true
4,14975,12942,12,7.2,18.900000,1,false,false,41.000000,7750000.0,1980,4,55.603920,37.726772,2.48,251,16,true


In [7]:
data.describe

<bound method NDFrame.describe of            id  building_id  floor  kitchen_area  living_area  rooms  \
0       14971        13921      9           8.5    21.000000      1   
1       14972        10161      8           8.8    32.599998      2   
2       14973         7378      2           9.5    21.100000      1   
3       14974        10856     10          10.0    20.000000      1   
4       14975        12942     12           7.2    18.900000      1   
...       ...          ...    ...           ...          ...    ...   
141357  87995        23071     13          10.0    24.000000      2   
141358  87996         3208      1           8.0    19.299999      1   
141359  87997        20199      2          10.2    61.700001      4   
141360  87998        19861      2          12.9    44.799999      3   
141361  87999        11349      1           6.0    27.799999      2   

       is_apartment studio  total_area       price  build_year  \
0             false  false   41.000000   760000

Duplicates

In [8]:
features = data.drop(columns='id').columns.tolist()

In [9]:
is_duplicated = data.duplicated(subset=features, keep=False)

In [10]:
data = data[~is_duplicated].reset_index(drop=True)

Missing values

In [23]:
col_with_nans = data.isnull().sum()

In [24]:
col_with_nans

id                       0
building_id              0
floor                    0
kitchen_area         10271
living_area          15719
rooms                    0
is_apartment             0
studio                   0
total_area               0
price                    0
build_year               0
building_type_int        0
latitude                 0
longitude                0
ceiling_height           0
flats_count              0
floors_total             0
has_elevator             0
dtype: int64

There is no nans but only for now 
(there will be new data)

In [28]:
col_with_nans = col_with_nans[col_with_nans > 0].index

In [29]:
col_with_nans

Index(['kitchen_area', 'living_area'], dtype='object')

In [30]:
for col in col_with_nans:
    if data[col].dtype in ['float', 'int']:
        fill_value = data[col].mean()
    if data[col].dtype == 'object':
        fill_value = data[col].mode().iloc[0]
    data[col] = data[col].fillna(fill_value)

0 values instead of null

In [19]:
non_zero_features = ['kitchen_area', 'living_area', 'rooms', 'total_area', 'price', 'build_year', 'ceiling_height', 
                     'flats_count', 'floors_total']

In [31]:
data[non_zero_features].eq(0).any()

kitchen_area      False
living_area       False
rooms             False
total_area        False
price             False
build_year        False
ceiling_height    False
flats_count       False
floors_total      False
dtype: bool

In [32]:
data[non_zero_features].eq(0).sum()

kitchen_area      0
living_area       0
rooms             0
total_area        0
price             0
build_year        0
ceiling_height    0
flats_count       0
floors_total      0
dtype: int64

A lot of 0 in kitchen and living area.
Let's change them for nans (in dag it will be before work with nans)

In [22]:
data[non_zero_features] = data[non_zero_features].replace(0, float('nan'))

Outliers

In [ ]:
num_cols = data.select_dtypes(['float']).columns

In [35]:
threshold = 1.5
potential_outliers = pd.DataFrame()

for col in num_cols:
    Q1 = data[col].describe().loc['25%']
    Q3 = data[col].describe().loc['75%']
    IQR = Q3 - Q1
    margin = IQR * threshold
    down = Q1 - margin
    up = Q3 + margin
    potential_outliers[col] = ~data[col].between(down, up)

In [38]:
outliers = potential_outliers.any(axis=1)

In [44]:
data = data[~outliers]